# SAD VAE

In [ ]:
(Super Awesome Dense-layer)

In [1]:
%whos

Interactive namespace is empty.


In [2]:
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
from sklearn.model_selection import train_test_split
from typing import Tuple
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import os

In [3]:
eight_by_five = pd.read_csv(
    '/Users/karlberb/work/sample_count/smpl_gen/architechture/X_intersect_5k_mad_feats_cohorts_n25.tsv',
    sep = '\t', index_col = 0)

In [5]:
eight_by_five

,N:GEXP::COL1A1:1277:,N:GEXP::ALB:213:,N:GEXP::FN1:2335:,N:GEXP::GAPDH:2597:,N:GEXP::ACTB:60:,N:GEXP::KRT5:3852:,N:GEXP::FTL:2512:,N:GEXP::EEF1A1:1915:,N:GEXP::B2M:567:,N:GEXP::COL3A1:1281:,...,N:GEXP::SMURF1:57154:,N:GEXP::GNPNAT1:64841:,N:GEXP::DMXL1:1657:,N:GEXP::AKT3:10000:,N:GEXP::ProSAPiP1:9762:,N:GEXP::RANBP3:8498:,N:GEXP::SIDT2:51092:,N:GEXP::AS3MT:57412:,N:GEXP::ALOX5:240:,N:GEXP::PHF2:5253:
TCGA-OR-A5J1,485.59,1.4409,1558.60,57986.0,68807.0,0.0000,13570.0,33046.0,18943.0,325.65,...,2448.60,383.77,640.73,853.03,243.04,3165.2,1013.50,133.53,90.7780,1230.10
TCGA-OR-A5J2,4269.10,5.2342,20086.00,45123.0,71065.0,0.8053,59215.0,61616.0,17621.0,13904.00,...,1632.40,525.03,801.24,678.84,501.27,1733.7,937.23,1985.00,26.5740,1115.70
TCGA-OR-A5J3,1164.90,5.3325,7744.60,271000.0,73505.0,1.1850,56320.0,77490.0,29841.0,1450.50,...,1785.50,674.86,1148.30,250.04,267.22,3143.2,1407.90,16851.00,20.1450,798.70
TCGA-OR-A5J5,1531.40,312.9400,17322.00,127710.0,111060.0,0.0000,40738.0,40610.0,6510.5,749.03,...,1209.10,254.07,1507.40,469.40,307.51,5023.2,617.51,26278.00,61.1930,2089.10
TCGA-OR-A5J6,376.11,1.1177,787.43,45653.0,52458.0,0.0000,166900.0,79786.0,39353.0,415.23,...,805.25,245.90,810.34,357.67,829.90,2113.6,2382.70,15389.00,156.4800,452.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WC-AA9E,76.65,0.8861,403.63,56811.0,69981.0,30.5720,37423.0,98720.0,16375.0,922.02,...,1122.70,1078.40,621.62,636.24,587.50,1790.4,2318.20,381.48,4.4307,1086.00
TCGA-YZ-A980,1841.30,0.8220,2852.40,103200.0,44996.0,1.2330,48825.0,101630.0,29023.0,1208.40,...,815.75,254.42,886.97,958.90,3026.30,2503.9,993.33,248.25,71.9280,1382.70
TCGA-YZ-A982,144.25,0.0000,628.32,59826.0,82306.0,7.0796,73261.0,64212.0,7694.7,152.21,...,924.24,233.63,142.48,859.29,734.51,2144.2,1912.80,363.70,23.0090,1319.50
TCGA-YZ-A983,490.15,0.6158,774.01,83361.0,37870.0,53.5710,32539.0,348050.0,10348.0,592.98,...,1221.20,489.53,900.25,364.53,742.61,1456.9,1181.20,100.98,21.5520,942.73


In [4]:
ebf_y = pd.read_csv(
    '/Users/karlberb/work/sample_count/smpl_gen/architechture/y_intersect_5k_mad_feats_cohorts_n25.tsv',
    sep = '\t', index_col = 0)

In [6]:
ebf_y

,Labels
TCGA-OR-A5J1,ACC_2
TCGA-OR-A5J2,ACC_2
TCGA-OR-A5J3,ACC_3
TCGA-OR-A5J5,ACC_3
TCGA-OR-A5J6,ACC_1
...,...
TCGA-WC-AA9E,UVM_3
TCGA-YZ-A980,UVM_1
TCGA-YZ-A982,UVM_2
TCGA-YZ-A983,UVM_3


### Normalization

In [3]:
def normalize(data: pd.DataFrame, features: list, feature_range: Tuple = None, scaler=None) -> (pd.DataFrame, any):
        if feature_range is not None and scaler is None:
            scaler = MinMaxScaler(feature_range=feature_range)
            scaler.fit(data)
        elif feature_range is None and scaler is None:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            scaler.fit(data)

        data = scaler.transform(data)

        return pd.DataFrame(columns=features, data=data), scaler

In [ ]:
scaler = MinMaxScaler(eight_by_five)

In [8]:
%whos

Variable           Type          Data/Info
------------------------------------------
CSVLogger          type          <class 'tensorflow.python<...>ras.callbacks.CSVLogger'>
EarlyStopping      type          <class 'tensorflow.python<...>callbacks.EarlyStopping'>
MinMaxScaler       type          <class 'sklearn.preproces<...>sing._data.MinMaxScaler'>
Model              type          <class 'tensorflow.python<...>s.engine.training.Model'>
StandardScaler     type          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
Tuple              _TupleType    typing.Tuple
ebf_y              DataFrame                  Labels\nTCGA<...>\n[8009 rows x 1 columns]
eight_by_five      DataFrame                   N:GEXP::COL<...>8009 rows x 5000 columns]
keras              module        <module 'tensorflow.keras<...>rflow/keras/__init__.py'>
layers             module        <module 'tensorflow.keras<...>eras/layers/__init__.py'>
normalize          function      <function normalize at 0x7ff71e45f

### Split handler

In [4]:
def create_splits(input_data: pd.DataFrame, without_val: bool = False) -> Tuple:
        """
        Creates train val test split of the data provided
        @param input_data: The input data which should be split
        @param without_val: If true only a test and a train set will be created
        @return: A tuple containing all the sets.
        """

        # No validation set will be created
        if without_val:
            return train_test_split(input_data, test_size=0.2, random_state=1, shuffle=True)

        # Create validation set
        X_dev, X_val = train_test_split(input_data, test_size=0.05, random_state=1, shuffle=True)
        X_train, X_test = train_test_split(X_dev, test_size=0.25, random_state=1, shuffle=True)
        return X_train, X_val, X_test

### Create custom sampling layer

In [5]:
class Sampling(layers.Layer): # inheriting
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs): # probably a tuple
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Create VAE 

In [6]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder: Model = encoder
        self.decoder: Model = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss_fn = keras.losses.MeanSquaredError()
            reconstruction_loss = reconstruction_loss_fn(data, reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        return self.decoder(z)

In [ ]:
# VAE class has its own call, same name from sampling
# within a class the 'def' is defining a method
# Namespaces, local, global

### Adjust hyperparameters

In [7]:
latent_dim = 500 # Latent dims
data_path = "/Users/karlberb/work/sample_count/smpl_gen/architechture/X_intersect_5k_mad_feats_cohorts_n25.tsv"
# Path to the data
# epochs = 100 # The number of epochs
batch_size = 128 # The batch size
base_path = "decoded_BRCA" # The base path where results are being stored
prefix = "v00" # The prefix used for result files

In [13]:
pwd

'/Users/karlberb/work/sample_count/smpl_gen/VAE-Subtype-Sample-Generation'

In [14]:
ls

SAD_VAE.ipynb                       library/
SAD_VAE_00_bk.ipynb                 local_diagnostics_2022-08-11.ipynb
SAD_VAE_01_bk.ipynb                 loss_plot_template.py
UMAP/                               output_reports/
VAE_dense_layer.py                  readme.md
VAE_rvrs_engr_02.ipynb              requirements.txt
VAE_rvrs_engr_summary.ipynb         sample_gen.py
data/                               sample_gen.sh
decoded_BRCA/                       sample_gen_data.sh
error_reports/                      sample_generator.py
fbrctd.sh                           script_archive/
fbrctd_00_results/                  venv/
image_gen/


In [12]:
%whos

Variable           Type          Data/Info
------------------------------------------
CSVLogger          type          <class 'tensorflow.python<...>ras.callbacks.CSVLogger'>
EarlyStopping      type          <class 'tensorflow.python<...>callbacks.EarlyStopping'>
MinMaxScaler       type          <class 'sklearn.preproces<...>sing._data.MinMaxScaler'>
Model              type          <class 'tensorflow.python<...>s.engine.training.Model'>
Sampling           type          <class '__main__.Sampling'>
StandardScaler     type          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
Tuple              _TupleType    typing.Tuple
VAE                type          <class '__main__.VAE'>
create_splits      function      <function create_splits at 0x7ff71e45faf0>
ebf_y              DataFrame                  Labels\nTCGA<...>\n[8009 rows x 1 columns]
eight_by_five      DataFrame                   N:GEXP::COL<...>8009 rows x 5000 columns]
keras              module        <module 'tensorflow

### Load data

In [8]:
data = pd.read_csv(data_path, sep='\t', index_col=0)
# truncate everything after the 4th decimal
data = np.trunc(1000 * data) / 1000

train_data, val_data, test_data = create_splits(input_data=data, without_val=False)

train_data, scaler = normalize(train_data, features=train_data.columns)
val_data, _ = normalize(val_data, features=val_data.columns, scaler=scaler)
test_data, _ = normalize(data=test_data, features=test_data.columns, scaler=scaler)

In [9]:
train_data

,N:GEXP::COL1A1:1277:,N:GEXP::ALB:213:,N:GEXP::FN1:2335:,N:GEXP::GAPDH:2597:,N:GEXP::ACTB:60:,N:GEXP::KRT5:3852:,N:GEXP::FTL:2512:,N:GEXP::EEF1A1:1915:,N:GEXP::B2M:567:,N:GEXP::COL3A1:1281:,...,N:GEXP::SMURF1:57154:,N:GEXP::GNPNAT1:64841:,N:GEXP::DMXL1:1657:,N:GEXP::AKT3:10000:,N:GEXP::ProSAPiP1:9762:,N:GEXP::RANBP3:8498:,N:GEXP::SIDT2:51092:,N:GEXP::AS3MT:57412:,N:GEXP::ALOX5:240:,N:GEXP::PHF2:5253:
0,-0.947297,-1.000000,-0.969799,-0.793818,-0.732542,-0.341712,-0.976573,-0.605559,-0.911924,-0.949642,...,-0.755035,-0.879660,-0.903399,-0.973896,-0.779143,-0.241858,-0.939249,-0.955923,-0.982767,-0.609653
1,-0.995571,-1.000000,-0.985476,-0.924626,-0.899130,-0.999999,-0.973177,-0.641519,-0.225584,-0.994590,...,-0.641336,-0.466613,-0.773139,-0.907012,-0.886528,-0.723126,-0.782280,-0.797045,-0.980717,-0.762687
2,-0.984093,-1.000000,-0.997276,-0.906936,-0.369010,-0.999995,-0.958580,-0.751580,-0.986312,-0.989380,...,-0.902831,-0.731008,-0.951009,-0.991683,-0.663736,-0.456342,-0.858367,-0.995285,-0.974354,-0.597461
3,-0.999373,-1.000000,-0.997380,-0.907137,-0.792348,-1.000000,-0.944637,-0.834890,-0.522519,-0.998972,...,-0.642856,-0.566864,-0.902944,-0.727004,-0.577066,-0.375568,-0.828012,-0.951261,-0.998246,-0.368659
4,-0.982041,-1.000000,-0.993113,-0.688988,-0.793506,-0.994542,-0.946774,-0.254113,-0.932917,-0.985750,...,-0.960689,-0.788804,-0.978246,-0.978289,-0.810980,-0.725610,-0.832360,-0.955886,-0.989144,-0.413375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5701,-0.998256,-0.999998,-0.871050,-0.966357,-0.953986,-1.000000,-0.987255,-0.481600,-0.795452,-0.996509,...,-0.930664,-0.950256,-0.869018,-0.776425,-0.772646,-0.437966,-0.717059,-0.979239,-0.977441,-0.058837
5702,-0.855816,-0.999995,-0.971256,-0.908756,-0.276355,-0.999323,-0.803458,-0.628528,-0.325899,-0.908656,...,-0.938192,-0.808332,-0.960468,-0.970899,-0.977138,-0.459264,-0.816054,-0.992742,-0.917716,-0.910294
5703,-0.984377,-0.999967,-0.979959,-0.323880,-0.664782,-0.999995,-0.857306,-0.585576,-0.796923,-0.993890,...,-0.884626,-0.953229,-0.884812,-0.923095,-0.955131,-0.596664,-0.709450,-0.915148,-0.782433,-0.618835
5704,-0.955419,-0.999998,-0.899891,-0.836295,-0.608676,-0.999889,-0.830999,0.087236,0.213301,-0.920000,...,-0.887260,-0.595848,-0.880112,-0.910723,-0.906377,-0.782411,-0.847539,-0.985615,-0.705015,-0.742020


In [22]:
train_data.shape

(5706, 5000)

In [20]:
val_data.shape

(401, 5000)

In [21]:
test_data.shape

(1902, 5000)

### Build model

In [23]:
train_data.shape[1]

5000

In [29]:
input_dimensions

5000

In [12]:
epochs = 15 # The number of epochs

In [13]:
input_dimensions = train_data.shape[1]

# features count in first
encoder_inputs = keras.Input(shape=(input_dimensions,)) # <--- Please explain

x = layers.Dense(units=1000, activation="relu")(encoder_inputs)
x = layers.Dense(units=500, activation="relu")(x)
x = layers.Dense(units=100, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var]) # Sampling
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(units=100, activation="relu")(latent_inputs)
# x = layers.Dense(units=500, activation="relu")(x)
x = layers.Dense(units=1000, activation="relu")(latent_inputs)

decoder_outputs = layers.Dense(units=input_dimensions, activation="relu")(x) # 

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

vae: VAE = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

# vae.summary()

callbacks = []

early_stop = EarlyStopping(monitor="reconstruction_loss",
                           mode="min", patience=5,
                           restore_best_weights=True)
callbacks.append(early_stop)

csv_logger = CSVLogger(os.path.join(base_path, 'training.log'),
                       separator='\t')
callbacks.append(csv_logger)

history = vae.fit(train_data,
                  callbacks=callbacks,
                  validation_data=(val_data, val_data),
                  epochs=epochs, batch_size=batch_size)


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 5000)]       0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 1000)         5001000     input_5[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 500)          500500      dense_10[0][0]                   
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 100)          50100       dense_11[0][0]                   
____________________________________________________________________________________________

2022-09-30 14:25:45.262843: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
45/45 [==============================] - 8s 146ms/step - loss: 21.2220 - reconstruction_loss: 0.7071 - kl_loss: 6.1285 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 2/15
45/45 [==============================] - 6s 134ms/step - loss: 0.7052 - reconstruction_loss: 0.6986 - kl_loss: 0.0054 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 3/15
45/45 [==============================] - 6s 133ms/step - loss: 0.7010 - reconstruction_loss: 0.6986 - kl_loss: 0.0019 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 4/15
45/45 [==============================] - 6s 141ms/step - loss: 0.6993 - reconstruction_loss: 0.6986 - kl_loss: 6.2198e-04 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 5/15
45/45 [==============================] - 6s 135ms/step - loss: 0.6987 - reconstruction_loss: 0.6

In [14]:
z_mean, z_var, embedding = vae.encoder.predict(train_data)

In [15]:
embedding

array([[ 1.1393836 ,  0.37125504, -0.9112674 , ...,  0.6503524 ,
        -0.8516645 ,  1.1269784 ],
       [ 1.8653744 , -0.6948046 , -0.50953454, ...,  0.6189342 ,
        -1.5743269 , -0.26229087],
       [ 0.45815405,  1.0232786 , -1.3403239 , ..., -0.7619574 ,
        -1.8295772 , -2.6909783 ],
       ...,
       [ 0.87550706, -1.2188705 ,  0.7291777 , ..., -0.962931  ,
         2.661737  ,  1.3980938 ],
       [ 1.6765723 , -0.27009642, -1.3959305 , ..., -0.06532443,
         1.9460304 ,  0.46898675],
       [-0.7611093 ,  0.04697263,  0.5281616 , ...,  1.5470366 ,
         0.9485749 ,  0.2517841 ]], dtype=float32)

In [16]:
decoder.predict(embedding)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [26]:
vae: VAE = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [27]:
epochs = 15 # The number of epochs

In [ ]:
history = vae.fit(train_data,
                  callbacks=callbacks,
                  validation_data=(val_data, val_data),
                  epochs=epochs, batch_size=batch_size)

In [28]:
history = vae.fit(train_data,
                  callbacks=callbacks,
                  validation_data=(val_data, val_data),
                  epochs=epochs, batch_size=batch_size)

Epoch 1/15
45/45 [==============================] - 22s 471ms/step - loss: 0.6995 - reconstruction_loss: 0.6986 - kl_loss: 2.2449e-04 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 2/15
45/45 [==============================] - 20s 455ms/step - loss: 0.6985 - reconstruction_loss: 0.6987 - kl_loss: 8.4374e-07 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 3/15
45/45 [==============================] - 21s 465ms/step - loss: 0.6986 - reconstruction_loss: 0.6986 - kl_loss: 0.0000e+00 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 4/15
45/45 [==============================] - 21s 459ms/step - loss: 0.6986 - reconstruction_loss: 0.6986 - kl_loss: 0.0000e+00 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 5/15
45/45 [==============================] - 21s 464ms/step - loss: 0.6986 - reconstr

### Create embeddings for test data

In [39]:
z_mean, z_var, embedding = vae.encoder.predict(test_data)

In [40]:
embedding

array([[ 1.9870646 ,  1.0416583 , -0.7838087 , ..., -0.53242105,
        -0.7526022 , -0.39004812],
       [ 0.34480008, -1.2102859 ,  0.516639  , ..., -0.17086384,
        -0.87621814, -0.41115296],
       [ 0.9989898 , -1.5573266 ,  1.3649637 , ..., -1.0215573 ,
        -0.10033275,  1.0939907 ],
       ...,
       [-1.1655518 , -0.5408735 ,  0.41010308, ...,  0.9028942 ,
         1.2008036 ,  1.0852925 ],
       [ 2.0437467 ,  1.3344986 ,  0.0473986 , ..., -1.1208513 ,
         0.86585915,  1.5747652 ],
       [-0.9168553 , -1.058427  ,  0.9859297 , ..., -0.4353223 ,
         1.3822342 , -0.11381175]], dtype=float32)

In [41]:
decoder.predict(embedding)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
z_mean, z_var, embedding = vae.encoder.predict(test_data)
embedding = pd.DataFrame(embedding)
embedding.to_csv(Path(base_path, f"{prefix}_embeddings.csv"), index=False)